## Overview

This notebook explores the use of Multi Armed bandits to arrive at the most favourable design for an online experiment. Online experiment involves multiple design choice that can be shown to user to get value, this value can be revenue obtained once the user interacts an the online service. A typical example can be the design of the ad, that can make it more suitable for the users to click on the ad thereby generating revenue for the organisation. 

Typically revenue comes later in the user funnel when user make as a purchase but to understand the impact of different strategies we will assume that clicking on the ad generates a fixed revenue per user to the organization. An example of this could be Google running an A/B/n test to improve the interface for the user making it more desirable for a user to click on the ad, the event that generates revenue for every click.

Overall process of designing and evaluating an A/B test can be broadly divided into three components, that are as follows - 

**1. Design choices** 

In real world, these can be different marketing strategy for promotions that a company want to test before rolling out to the masses, or different choices of UI changes that a company want to test out to understand what choice leads to a higher engagement with the audience. In this example we will assume there are 4 different ad designs that the creative team has come up with and the management wants to run an A/B/n test to arrive at the best performing ad design that can be rolled out enterprise wide.

**2. Design selection strategy**

Design selection strategy govers what ad to display to a group of audience, this can be done in two ways - 1. Random assignment, and 2. Bandit algorithms. Random assignments allocates the traffic equally to all the possible choices during the test and once test is done, we evaluate the performance based on the KPI of concern (could be clicks or revenue). Though this is a pretty straigthforwad strategy to implement, it doesn't take into account the information obtained during the test itself to divert the audience from less desirable choices to more desirable choices. For example, let's say we have four designs that we want to test, and a total of 1 million user impressions is selected as stopping criteria (equivalent to approx 250k impressions for each design choice) for the test. In this situtation, there coould be a possibility that way before the test ends, the early performance can indicate that one of the choice performs significantly worse that the rest of choices, with random assignment there is no way that the rest of the audience of that choice can be diverted to more desirable ones, that's when Bandit algorithms come into play. With exploration-exploitation approach, bandit algorithms wait for getting the right amount of data to make a decision to cut off the audience flow to the undesirable design.

**3. Performance evaluation and Optimal Design**

The end goal of any A/B test is to understand the optimal choice by takeing feedback from users on a small scale. To arrive at the optimal design choice at the end of test, one looks at KPI or a set of KPIs, also called goal metrics, that the company wants to drive and evaluates which choice gives the most impact. The most common way of doing this to perform hypothesis testing, assume that all choice affect the goal metrics similarly and there is no statistical difference in all the design choice and this will be the null hypothesis. Based on the observed data during the test, distribution of goal metrics can be emprically obtained to reject the null hypothesis and optimal design choice can be selected as the one with best performing goal metric.


*Note: Designing and evaluating an A/B test is not that easy in real world, there are a lot of considerations that need to be taken and continuous monitoring of the test is done to avoid driving spurious and incorrect conclusion in the end. For the sake of simiplicity, in this simulation exercise we will focus only on bandit algorithm for experimental design and not address the real world challenges.*

## Bandit algorithms

Multi-arm bandit problem refers to problem space where there are a bunch of possible choices and a decision need to taken (either once or multiplie time) to maximize some reward despite not knowing what the reward distribution of each choices looks like, these kinds of problem are solved with some exploration-exploitation approach where initially exploration is done to get some idea about the reward pattern of each choices and once enough information is obtained the best choice is exploiting more frequently with still some exploration to ensure the right choice is being exploited.

In this section, we will talk about some of the common bandit algorithms with a brief description of how they work. Other than random assignment we will consider 4 different bandit algorithm to dynamically allocate traffic to different design choices during the experiment. Theoretical evaluation of these algorithm is already available in literature, where regret bounds of different bandits algorithm are evaluated analytically (regret being the lost opportunity in the absence of knowledge about the optimal choice). For this simulation, we will evaluate bandit algorithms by empirically calculating regret since the goal metric's distribution of each design choice will be known but hidden to algorithms during the test.

Multi arm bandit terminology from ad selection experiment - 
* trial - showing up an ad to the user
* arm - design choices considered during experimentation
* reward - goal metrics, click through rate

**Epsilon-greedy**

Epsilon-greedy approach explicitly tends to balance between exploration and exploitation during a trial based on a probability factor $\epsilon$. Assume that for each trial a hypothetical coin, that has a probability $\epsilon$ of getting head, will be tossed, everytime head comes the arm will be choosen at random (exploration) but if tails come up then the arm with highest mean reward will be selected. With a fixed $\epsilon$, at any point in time in the experiment, exploration will be fixed, but that might not be the most optimal approach, since at the beginning of the experiment we would want to explore more to get information about the rewards but as we progress we can have enough data to ascertain the best arm, thereby reducing the need for exploration. To tackle this behaviour during the test, annealing can be used where the parameter $\epsilon$ will be a function of time, starting with a high number and gradually reducing as more and more information gets available.


**UCB**

UCB, short for, Upper confidence bound, is another approach for adaptive exploration known as optimism under uncertainty, that manages exploration vs exploitation dilemma by setting the upper uncertainity bounds on each action taken in the past. Consider that after $T$ trails, every action possible (arm pull) $a_{i}$ is selected $N_{i}$ times and average reward $Q_{T, i}$ obtained whenever an action is selected in the past, using this information an upper bound on reward can be estimated. The goal is to always select the arm that has the highest upper condidence bound, that gets updated after the action is taken and outcome is known. Though for the start of the experiment, each arm will be selected randomly $k$ times to have enough information to estimate the upper bound. As the experiment continues, for every trial, the upper confidence bound of the selected arm $a_{i}$ will be updated based on the reward obtained. After a period of time, the arms that constantly underperforms will have reduced probabity of getting selected ensuring the best possible action is taken given the information available. 


**Thompson sampling**

Compared to other multi arm bandit approaches described above, where either the mean reward or upper bound on reward is used as a criteria to select the best arm for a trial, Thompson sampling creates a distribution and samples from the distribution to identify the best arm. Also, for every trial, the distribution of selected arm is updated based on observed reward. For a bernoulli bandit case, such as ours where the reward is binary (either a click is observed or not after the arm selection), a beta distribution is assumed as the prior and the parameters of the distribution are updated as an arm is selected and reward is observed with a prior of each arm having the same and an equal likelihood of getting a click. Assuming a beta distribution simplifies the calculation as beta distribution is its own conjugate prior, hence after each trial the prior and posterior distribution family is same and only the parameters $\a$ and $\b$ needs to be changed.

Beta distribution - $\frac{\Gamma(a + b)}{\Gamma(a)\Gamma(b)}x^{a - 1}(1 - x)^{b - 1}$

For an arm selected @ t, arm $a_{i}$, if $r{t, i}$ = 1, $a = a + 1$, else $b = b + 1$

The parameters, $a$ and $b$ can be though of number of times the trial is won (click is observed) and lost (click is not observe) respectively


## Simulation Design

To understand the efficacy of the above mentioned multi arm bandit approaches, simulation will be setup where the true reward distribution of each arm will be defined so that exact regret or opportunity loss due to not knowing the best action can be calculated.

*Environment setup*
1. Arms or design choices - Defining the choices of ad designs (that will be later referred to as arms) with reward (click through rate and revenue) unknown
2. Bandit algorithms - 
    * $\epsilon$ greedy with linear annealing
    * UCB
    * Thompson sampling
    
*Stopping criteria* - After the specified number of trails are completed

*Evaluation*
1. Evaluating the performance of dfferent algorithms on picking the best arm
2. Cumulative total regret for the complete test duration
3. Total number of trials required by each approach to arrive at the best arm
4. Incremental revenue compared to baseline (random assignment)

*Assumptions*
1. Uniform (stratified audience)
2. Ad selection only depends on the choice presented to the user
3. No temporal change in user behaviour when the experiment is running


## Simulation

In [4]:
import numpy as np
from scipy.stats import bernoulli, beta, binom, norm
import matplotlib.pyplot as plt
from typing import List, Dict
import pandas as pd
from mab_algos.multi_arm_bandits import *

ModuleNotFoundError: No module named 'mab_algos'

### Design choices (Arms)

Lets assume there are 4 different choices of ad that can be shown to the user, these ads will have inherent reward -> click through probability $p_{i}$ that will be know by design but will be unknown during the experiment. For setting up a limit to the length of online experiment it is assumed that the experiment will terminate after ads are shown 10,000 times.

In [ ]:
class Arm:
    '''Arm object with defined click through probability'''
    def __init__(self, p: float) -> None:
        # bernoulli distribution with mean, and variance given the click through rate
        self.mu = p
        self.var = p * (1 - p)
        
    def reward(self, pulls: int=1) -> float:
        return list(bernoulli.rvs(self.mu, size=pulls))
    
class ArmsList(List[Arm]):
    
    def __init__(self, n_arms: int, p_arms: List[float]) -> None:
        self.n_arms = n_arms
        self.arms = []
        for i in range(n_arms):
            self.arms.append(Arm(p_arms[i]))
            
    def arms_reward(self, selected_arm: int, pulls: int=1) -> float:
        return self.arms[selected_arm].reward(pulls)
    
    def get_best_arm(self) -> Arm:
        best_arm = self.arms[0]
        for arm in self.arms:
            if arm.mu > best_arm.mu:
                best_arm = arm
        return best_arm
    
    # TODO: add a function to track the selection of different arms over time

In [ ]:
# assuming fixed but unknown ctr distribution of the the design choices
n_arms, n_trials = 4, 10000
np.random.seed(40)
ctr_list = [0.02, 0.03, 0.04, 0.044]
arms_list = ArmsList(n_arms=n_arms, p_arms=ctr_list)
arms_list.get_best_arm().mu

### Util functions

In [ ]:
def plot_rewards(arms_list: ArmsList, rewards_list: List) -> None:
    n_arms = arms_list.n_arms
    arm_label = ['Arm' + str(i+1) for i in np.arange(n_arms)]
    
    actual_mean_rewards = [arm.mu for arm in arms_list.arms]
    actual_std_rewards = [np.sqrt(arm.var) for arm in arms_list.arms]
    mean_rewards = [np.mean(rewards_list[i]) for i in np.arange(n_arms)]
    stddev_rewards = [np.std(rewards_list[i])/np.sqrt(len(rewards_list[i])) for i in np.arange(n_arms)]
    
    plt.errorbar(arm_label, mean_rewards, stddev_rewards, linestyle='None', label='estimated', marker = 'o')
    plt.errorbar(arm_label, actual_mean_rewards, linestyle='None', label='actual', marker='^')
    plt.legend()
    plt.show()

### Random Assignment

In [ ]:
rewards_list = []
np.random.seed(0)
pulls_per_arm = int(n_trials/n_arms)

for i in range(n_arms):
    rewards_list.append(
        arms_list.arms_reward(selected_arm=i, pulls=pulls_per_arm)
    )

plot_rewards(arms_list, rewards_list)

### $\epsilon$ greedy

**10,000 trials distributed over the 4 design choices, with linear annealing starting from 100th trial**

In [ ]:
from abc import ABC, abstractmethod

class MAB(ABC):
    def __init__(self, n_arms: int, n_trials: int) -> None:
        self.n_arms = n_arms
        self.n_trials = n_trials
        self.selected_arms = np.ones(n_trials)*-1
        self.rewards_list = {}
        
        for j in range(n_arms):
            self.rewards_list[j] = [0]
            
    def get_best_arm(self) -> int:
        best_arm = np.argmax([sum(self.rewards_list[i])/len(self.rewards_list[i]) for i in range(self.n_arms)])
        return best_arm
        
    def get_selected_arms(self) -> np.array:
        return self.selected_arms
            
    @abstractmethod
    def run_test(self)  -> Dict:
        raise NotImplemented
    
    def plot_arms_distribution(self) -> None:
        pd.Series(self.selected_arms).hist()

class EpsGreedy(MAB):
    def __init__(self, n_arms: int, n_trials: int=10000, eps_init: float=0.05, buffer: int=100) -> None:
        super().__init__(n_arms, n_trials)
        self.eps_list = np.concatenate((
            np.array([0]*buffer),
            np.linspace(start=eps_init, stop=1, num=n_trials-buffer))) 
            
    def run_test(self, arms_list: ArmsList) -> Dict[int, List]:
        np.random.seed(100)
        for i in range(self.n_trials):
            eps = self.eps_list[i]
            if np.random.choice([1, 0], p=[eps, 1-eps]):
                best_arm = self.get_best_arm()
            else:
                best_arm = np.random.choice(self.n_arms)

            self.selected_arms[i] = best_arm
            self.rewards_list[best_arm] += arms_list.arms_reward(selected_arm=best_arm)

        return self.rewards_list

In [ ]:
eps_greedy = EpsGreedy(n_arms, 10000, 0.05, 100)
eps_greedy_reward = eps_greedy.run_test(arms_list)

In [ ]:
plot_rewards(arms_list, eps_greedy_reward)

In [ ]:
eps_greedy.plot_arms_distribution()

**Can confidence bound be estimated with less number of trials**

In [ ]:
# 1000 trials
eps_greedy = EpsGreedy(n_arms, 1000, 0.05, 100)
eps_greedy_reward = eps_greedy.run_test(arms_list)
plot_rewards(arms_list, eps_greedy_reward)

In [ ]:
# 2000 trials
eps_greedy = EpsGreedy(n_arms, 2000, 0.05, 100)
eps_greedy_reward = eps_greedy.run_test(arms_list)
plot_rewards(arms_list, eps_greedy_reward)

In [ ]:
# 5000 trials
eps_greedy = EpsGreedy(n_arms, 5000, 0.05, 100)
eps_greedy_reward = eps_greedy.run_test(arms_list)
plot_rewards(arms_list, eps_greedy_reward)

In [ ]:
# 1000 trials with 500 for priming
eps_greedy = EpsGreedy(n_arms, 1000, 0.05, 500)
eps_greedy_reward = eps_greedy.run_test(arms_list)
plot_rewards(arms_list, eps_greedy_reward)

In [ ]:
# 5000 trials with 500 for priming
eps_greedy = EpsGreedy(n_arms, 5000, 0.05, 500)
eps_greedy_reward = eps_greedy.run_test(arms_list)
plot_rewards(arms_list, eps_greedy_reward)

In [ ]:
# 10000 trials with 500 for priming
eps_greedy = EpsGreedy(n_arms, 10000, 0.05, 500)
eps_greedy_reward = eps_greedy.run_test(arms_list)
plot_rewards(arms_list, eps_greedy_reward)

### UCB

In [ ]:
import scipy.stats as st
class UCB(MAB):
    def __init__(self, n_arms: int, n_trials: int=10000, ci: float=0.95, buffer: int=100) -> None:
        super().__init__(n_arms, n_trials)
        self.buffer = buffer
        self.c = st.norm.interval(alpha=ci)[1]
        self.upper_bound_list = []
        
    def get_upper_bound(self, selected_arm, n) -> float:
        n_a = sum([1 for arm in self.selected_arms if arm==selected_arm])
        return np.sum(self.rewards_list[selected_arm])/n_a + self.c * np.sqrt(np.log(n)/n_a)

    def get_best_arm(self, n) -> int:
        return np.argmax([self.get_upper_bound(arm, n) for arm in range(self.n_arms)])
    
    def run_test(self, arms_list) -> Dict[int, List]:
        np.random.seed(100)
        for i in range(self.n_trials):
            if i < self.buffer:
                best_arm = np.random.choice(self.n_arms)
            else:
                best_arm = self.get_best_arm(i)
            
            self.selected_arms[i] = best_arm
            self.rewards_list[best_arm] += arms_list.arms_reward(selected_arm=best_arm)

        return self.rewards_list 

# TODO: check if the option with ucb[a_i] < ucb[a_j] for every j != i, needs to be removed

In [ ]:
ucb = UCB(n_arms, 10000, 0.95, 100)
ucb_reward = ucb.run_test(arms_list)

plot_rewards(arms_list, ucb_reward)

ucb.plot_arms_distribution()

In [ ]:
ucb = UCB(n_arms, 10000, 0.75, 100)
ucb_reward = ucb.run_test(arms_list)

plot_rewards(arms_list, ucb_reward)

ucb.plot_arms_distribution()

In [ ]:
ucb = UCB(n_arms, 10000, 0.99, 100)
ucb_reward = ucb.run_test(arms_list)

plot_rewards(arms_list, ucb_reward)

ucb.plot_arms_distribution()

### Thompson sampling

In [ ]:
class ArmEmpiricalDistribution:
    from scipy.stats import beta
    def __init__(self, a=1, b=1) -> None:
        self.a = a
        self.b = b
        self.distribution = beta(a, b)
        
    def sample_reward(self, n_trial: int=1) -> float:
        return self.distribution.rvs(size=n_trial)
    
    def update_distribution(self, reward: List):
        for r in reward:
            if r == 1:
                self.a += 1
            else:
                self.b += 1
        self.distribution = beta(self.a, self.b)
        
    def get_mean(self) -> float:
        return self.a/(self.a + self.b)



class ThompsonSampling(MAB):
    def __init__(self, n_arms: int, n_trials: int=10000) -> None:
        super().__init__(n_arms, n_trials)
        self.arms_distribution = {}
        for i in range(self.n_arms):
            self.arms_distribution[i] = ArmEmpiricalDistribution()

    def get_best_arm(self) -> int:
        sample_reward_list = [arm_distr.sample_reward() for arm_distr in self.arms_distribution.values()]
        return np.argmax(sample_reward_list)
    
    def run_test(self, arms_list: List) -> Dict[int, List]:
        np.random.seed(100)
        for i in range(self.n_trials):
            best_arm = self.get_best_arm()
            self.selected_arms[i] = best_arm
            
            reward = arms_list.arms_reward(selected_arm=best_arm)
            self.rewards_list[best_arm] += reward
            self.arms_distribution[best_arm].update_distribution(reward)

        return self.rewards_list 

In [ ]:
ts = ThompsonSampling(n_arms, 10000)
ts_reward = ts.run_test(arms_list)

plot_rewards(arms_list, ts_reward)

ts.plot_arms_distribution()

## Comparitive analysis

* How does different MAB approaches perform when getting the best arm?
* Does the expected return from the experiment matches the actual return of each arm?
* Does the empiral click through rate for each arm obtained statistically different?
* Are we able to identify the best arm considering statistical significant?

* Which approach gives us the least regret (clicks that could have been obtained if the best arm is known)?

In [ ]:
# TODO:
# Define function to compare the empirical and actual click through rate
# Define function for regret, takes arms reward_list and actual arm instance (in terms of click lost)
# Define function for statistical significance, if the best arm is best arm significantly
# Back track the number of trials needed by each approach to find statistically significant best arm

## Statistical power
How many time the experiment needs to run to get statistically significant distribution?
Do MAB get it quickly?

## Conclusion

Advantages of Multi-arm bandit algorithms compared to tradition A/B testing mechanism with random assignment of traffic for the whole duration of the test

## References - 
* https://towardsdatascience.com/beyond-a-b-testing-multi-armed-bandit-experiments-1493f709f804
* https://analytics.googleblog.com/2013/01/multi-armed-bandit-experiments.html